In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import copy
import matplotlib.pyplot as plt
import torchvision
from torchvision import models
from torchvision import datasets
import torchvision.transforms as transforms
import time
import os
#from sklearn.model_selection import cross_val_score
#from skorch import NeuralNetClassifier
#from efficientnet_pytorch import EfficientNet # still planning to use
print(torch.__version__)
torch.cuda.is_available()

In [ ]:
torch.manual_seed(123)

In [ ]:
data_dir = 'D:\Cats-and-Dogs-breed-classification\data'
#pre-processing:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),                                  #resize ảnh về kích thước 224x224
        transforms.RandomHorizontalFlip(),                                  #lật ảnh ngang dọc
        transforms.ToTensor(),                                              #chuyển đổi hình ảnh sang dạng tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  #chuẩn hóa giá trị tensor đầu vào theo giá trị trung bình và độ lệch chuẩn của bộ dữ liệu ImageNet
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
#tạo đối tượng image_folder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
#tạo dataloaders để tải sữ liệu
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)
class_names = image_datasets['train'].classes

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts) #load model
    return model

In [ ]:
'''device = torch.device('cpu')
model = models.resnet18(pretrained=True)             #khởi tạo model
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 38)
model.to(device)
criterion = nn.CrossEntropyLoss()
optmizer = optim.SGD(model.parameters(), lr=0.005)
step_lr_scheduler = lr_scheduler.StepLR(optmizer,step_size=7, gamma=0.1)

model = train_model(model, criterion, optmizer, step_lr_scheduler, num_epochs=10)'''

In [ ]:
'''# save the model so you can easily load later
# load with: model.load_state_dict(torch.load(PATH))
PATH = 'D:/Cats-and-Dogs-breed-classification/resnet18_3_classes.pth'
torch.save(model.state_dict(), PATH)'''

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 38)

In [ ]:
def return_model_answer(img_path):
    #INSANE casting to the model stupidly done by me in almost 1 day of work (it was my first day on PyTorch ok)
    #model.load_state_dict(torch.load('resnet18_38_classes.pth'))
    model.load_state_dict(torch.load('resnet18_3_classes.pth'))
    model.eval()
    import cv2
    from PIL import Image
    img = img_path
    img_test = Image.open(img)
    plt.imshow(img_test)
    plt.show()
    img_test = img_test.resize((224,224))
    img_test = np.array(img_test.getdata()).reshape(*img_test.size,3) #hình ảnh được chuyển đổi thành một mảng numpy và được thay đổi hình dạng thành (224, 224, 3)
    img_test = img_test/255                                           #các giá trị pixel được chuẩn hóa bằng cách chia cho 255
    img_test = img_test.transpose(2,0,1)                              #hình ảnh được chuyển đổi thành kích thước (3, 224, 224)
    img_test = img_test[None,:,:,:]                                   #hình ảnh được mở rộng thêm một chiều ở đầu, trở thành (1, 3, 224, 224)
    img = torch.from_numpy(img_test)                                  #mảng numpy của hình ảnh được chuyển đổi thành một tensor PyTorch
    #img= img.cuda() # Disabled since the model don't use gpu (my 1050ti 4gb can't handle it)
    img = img.float()
    # pass to CNN to recognize it
    outputs = model(img)
    _, predicted = torch.max(outputs, 1)
    return (format(class_names[predicted]))

In [ ]:
test_dir =  'D:/Cats-and-Dogs-breed-classification/data/test/dog'
images = os.listdir(test_dir)
count = 0
raw = 0
for image_name in images:
    name = test_dir + '/' + image_name
    answer = return_model_answer(name)
    print(answer)
    if answer == 'dog':
        count += 1
    raw += 1
print("Accuracy = ", ((count/raw)*100) )

In [ ]:
'''breed = 'cat' #just switch the breed and run the val tests
val_dir = 'D:/Cats-and-Dogs-breed-classification/data/test/'
count = 0
samples = 0
for i in range(1,51):
    try:
        #name = val_dir + breed + '/' + breed + '_' + str(i) + '.jpg'
        name = val_dir + breed + '/' + '_' + str(i) + '.jpg'
        answer = return_model_answer(name)
        print(answer)
        samples+=1
        if answer == name:
            count+=1
    except:
        pass
breed_accuracy ="%.2f" % ((count/samples)*100)
print(breed + ' accuracy: ' + breed_accuracy + '%')'''
